In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin


from xgboost import XGBClassifier
import xgboost as xgb

from datetime import datetime
from datetime import date

import re
import pickle

In [2]:
import nltk

In [3]:
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [4]:
import textblob
from textblob import TextBlob

In [5]:
from string import punctuation

In [6]:
import requests,json

import csv
from datetime import datetime

### read train and test data

In [7]:
df_train = pd.read_csv('Traindata_pilot_sample_v0_119parts.csv', engine='python')

In [8]:
df_test = pd.read_csv('Testdata_pilot_sample_v0_119parts.csv', engine='python')

#### use only vehicle features data

In [12]:
df_X = df_train[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']]

In [13]:
df_X_cat = df_train[['mdl_yr','veh_line_cd', 'eng_cd']]

In [14]:
df_X_cat['mdl_yr'] = df_X_cat['mdl_yr'].astype('object')

<ipython-input-14-6977494ceeb5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X_cat['mdl_yr'] = df_X_cat['mdl_yr'].astype('object')


In [15]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [16]:
ohe.fit(df_X_cat)
onehot_encoded_X = ohe.transform(df_X_cat)

In [19]:
cat_names = np.concatenate(ohe.categories_, axis = 0)

In [21]:
cat_df = pd.DataFrame(onehot_encoded_X, columns = cat_names)

#### include reliability model score

In [22]:
reliability_df = pd.read_csv("f_rate_WB_118month_poc3.csv", engine='python')

In [23]:
reliability_df.columns = ['prt_num', 'veh_line_cd', 'mdl_yr', 'tis_wsd', 'Avg_Weibull_Failure', 'f_rate']

In [24]:
data_reliab_pivot = reliability_df.pivot_table(index=['veh_line_cd', 'mdl_yr', 'tis_wsd'], columns='prt_num', values=['Avg_Weibull_Failure'], aggfunc='first')
        
reliability_df_t = data_reliab_pivot.fillna(0.0)

In [25]:
df1 = pd.merge(df_X, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

/usr/local/lib/python3.8/dist-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


#### combine categorical, numerical, and reliability score variables

In [27]:
df_X_dummy = pd.concat([cat_df, df_X[['tis_wsd','milge']], df1.iloc[:,5:]], axis=1)

#### create tfidf matrix

In [30]:
lookup_table = 'Misspellings_Correction_Synonyms_Lookup_Table_Master.xls'

In [31]:
syn_df=pd.read_excel(lookup_table)

In [32]:
syn_df = syn_df[syn_df['Review'] != "D"]

In [33]:
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [34]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',
                       "N":'n',
                       "V":'v',
                       "R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [35]:
def TextPreprocess(text):
    # Avoid mixed data type errors and apply str on all rows

    text = str(text).lower()

    #Remove HTML Tags
    text = re.sub('<[^>]*>', '', text)
    r1 = re.compile('<.*?>')
    text = re.sub(r1, ' ', text)
    # Remove any token with numbers and/or special characters

    r2 = re.compile(r'^[0-9\W\_]+$')
    text = re.sub(r2, ' ', text)

    #Removing Punctuations
    text = re.sub(r'[?|!|\'|"|#]',r'',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        

    # Tokenize all text data using nltk package

    text = word_tokenize(text)

    # Remove stop words
    stop = stopwords.words('english')
    stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', 'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

    # Combine default nltk stop list and custom stop list to generate a deduplicated list
    english_stops = list(set(stop+stop1))

    text = [item for item in text if item not in english_stops]

    # Remove special characters occuring as prefixes
    text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
    # Remove special characters occuring as suffixes

    text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

    # Replace special characters with spaces when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length

    text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

    # Replace special characters with null when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length

    text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]
    # Remove stop words after data cleansing steps to remove any residual stop words

    text = [item for item in text if item not in english_stops]

    # remove any numbers after data cleansing steps to remove any residual numeric values

    r3 = re.compile(r'^[0-9\W\_]+$')
    text = [i for i in text if not r3.match(i)]

    #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
    #spell=SpellChecker() 

    text = [syn_dict.get(str(i), str(i)) for i in text]

    # Convert the list to string before passing to lemmatization module
    text = ' '.join(text)

    # Perform Lemmatization on the cleaned up data before tfidf vectorization

    text = lemmatize_with_postag(text)

    return(text)

In [36]:
text_df = df_train[['cust_txt']]

In [37]:
text_df['process_txt']=text_df['cust_txt'].apply(lambda x: TextPreprocess(x))

<ipython-input-37-0addefb527f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['process_txt']=text_df['cust_txt'].apply(lambda x: TextPreprocess(x))


In [36]:
### convert null values of customer_txt into ' '
text_df.loc[text_df['process_txt'].isnull()==True,'process_txt'] = ' '

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df = 0.5, min_df = 0.0005)

In [39]:
tfidf_matrix = vectorizer.fit_transform(text_df['process_txt'])

In [40]:
tfidf_dense = tfidf_matrix.todense()

In [41]:
tfidf_features = vectorizer.get_feature_names()

In [43]:
tfidf_df = pd.DataFrame(tfidf_dense, columns = tfidf_features)

#### combine tfidf_df with df_X_dummy

In [44]:
df_X_dummy_1 = pd.concat([df_X_dummy, tfidf_df], axis=1)

In [45]:
df_X_dummy_1.shape

(338248, 2335)

#### create label encoded y variables

In [46]:
le = LabelEncoder()
le.fit(df_train['target'])
label_encoded_y = le.transform(df_train['target'])

### create train-eval split and train xgb model

In [47]:
X_train, X_eval, y_train, y_eval = train_test_split(df_X_dummy_1, label_encoded_y, test_size = 0.2, random_state = 11)

In [48]:
dtrain = xgb.DMatrix(X_train, y_train)
deval = xgb.DMatrix(X_eval, y_eval)

In [49]:
eval_set = [(dtrain, 'train'), (deval, 'eval')]

In [50]:
num_class = len(set(df_train['target']))
num_class

119

In [51]:
params = {
    "objective": "multi:softprob",
    "num_class": num_class,
    ##"eval_metric": "auc",
    "booster": "gbtree",
    "tree_method": "gpu_hist",
    "eta": 0.1,
    "max_depth": 4,
    "sub_sample": 1,
    "col_sample_bytree": 1,
    "verbosity": 0,    
    "seed": 11    
}

In [52]:
num_rounds = 100
early_stopping_rounds = 10

In [53]:
gbm = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-mlogloss:2.23553	eval-mlogloss:2.24838
[1]	train-mlogloss:2.05986	eval-mlogloss:2.07687
[2]	train-mlogloss:1.94286	eval-mlogloss:1.96183
[3]	train-mlogloss:1.84523	eval-mlogloss:1.86611
[4]	train-mlogloss:1.76547	eval-mlogloss:1.78887
[5]	train-mlogloss:1.69697	eval-mlogloss:1.72249
[6]	train-mlogloss:1.63533	eval-mlogloss:1.66272
[7]	train-mlogloss:1.57922	eval-mlogloss:1.60848
[8]	train-mlogloss:1.53002	eval-mlogloss:1.56063
[9]	train-mlogloss:1.48495	eval-mlogloss:1.51680
[10]	train-mlogloss:1.44493	eval-mlogloss:1.47852
[11]	train-mlogloss:1.40775	eval-mlogloss:1.44291
[12]	train-mlogloss:1.37360	eval-mlogloss:1.41015
[13]	train-mlogloss:1.34153	eval-mlogloss:1.37960
[14]	train-mlogloss:1.31254	eval-mlogloss:1.35212
[15]	train-mlogloss:1.28488	eval-mlogloss:1.32601
[16]	train-mlogloss:1.25944	eval-mlogloss:1.30197
[17]	train-mlogloss:1.23569	eval-mlogloss:1.27944
[18]	train-mlogloss:1.21348	eval-mlogloss:1.25852
[19]	train-mlogloss:1.19291	eval-mlogloss:1.23927
[20]	train

#### create X_test and y_test

In [54]:
df_X_test = df_test[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']]

In [55]:
df_X_test_cat = df_test[['mdl_yr','veh_line_cd', 'eng_cd']]

In [56]:
df_X_test_cat['mdl_yr'] = df_X_test_cat['mdl_yr'].astype('object')

<ipython-input-56-7c3c2d4bf448>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X_test_cat['mdl_yr'] = df_X_test_cat['mdl_yr'].astype('object')


In [57]:
onehot_encoded_X_test = ohe.transform(df_X_test_cat)

In [59]:
cat_df_test = pd.DataFrame(onehot_encoded_X_test, columns = cat_names)

In [60]:
df_with_reliability = pd.merge(df_X_test, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

/usr/local/lib/python3.8/dist-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [61]:
df_X_test_dummy = pd.concat([cat_df_test, df_X_test[['tis_wsd','milge']], df_with_reliability.iloc[:,5:]], axis=1)

In [63]:
text_df_test = df_test[['cust_txt']]

In [64]:
text_df_test['process_txt']=text_df_test['cust_txt'].apply(lambda x: TextPreprocess(x))

<ipython-input-64-cc2d279a5ae0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df_test['process_txt']=text_df_test['cust_txt'].apply(lambda x: TextPreprocess(x))


In [65]:
### convert null values of customer_txt into ' '
text_df_test.loc[text_df_test['process_txt'].isnull()==True,'process_txt'] = ' '

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### define new tfidf verctorizor with vocab from train and use it to generate test tfidf

In [66]:
vectorizer_test = TfidfVectorizer(ngram_range=(1,2), vocabulary=tfidf_features)

In [67]:
tfidf_matrix_test_1 = vectorizer_test.fit_transform(text_df_test['process_txt'])

In [69]:
tfidf_features_test_1 = vectorizer_test.get_feature_names()

In [72]:
tfidf_dense_test = tfidf_matrix_test_1.todense()

In [73]:
tfidf_df_test = pd.DataFrame(tfidf_dense_test, columns = tfidf_features_test_1)

#### combine tfidf_df_test with df_X_test_dummy

In [74]:
df_X_test_dummy_1 = pd.concat([df_X_test_dummy, tfidf_df_test], axis=1)

#### create label encoded y variables for the test set

In [76]:
label_encoded_y_test = le.transform(df_test['target'])

#### score the test set with the trained xgb model

In [77]:
probs = gbm.predict(xgb.DMatrix(df_X_test_dummy_1))

In [78]:
df_prob = pd.DataFrame(probs, columns=le.classes_)

In [79]:
## find the top3 parts
k = 3

# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds=[[le.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds=[item[::-1] for item in preds]

preds

df_top3 = pd.DataFrame(preds, columns=("Top1", "Top2", "Top3"))
df_top3.head()

Top1    Top2    Top3
0  10346  10V346  15K601
1  11002   10346   3B436
2   8678   3A674   10346
3  11002   10346  15K601
4  11002   10346  15K601

In [80]:
y_test_part = list(df_test['target'])
N = len(y_test_part)
N

85041

In [81]:
n_correct = 0
for i in range(N):
    if (df_top3.iloc[i, 0]==y_test_part[i]) or (df_top3.iloc[i, 1]==y_test_part[i]) or (df_top3.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.9050693195047095


In [82]:
n_correct_1 = 0
for i in range(N):
    if df_top3.iloc[i, 0]==y_test_part[i]:
        n_correct_1 += 1
        
print(n_correct_1/N)

0.7710398513658118


#### part-wise accuracy

In [84]:
from sklearn.metrics import accuracy_score, jaccard_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

In [85]:
accuracy_score(y_test_part, df_top3.iloc[:, 0])

0.7710398513658118

In [86]:
y_labels_count = df_test.groupby('target')['clm_key'].count().to_frame(name='row_count').sort_values('row_count', ascending=False).reset_index()

In [87]:
y_labels = y_labels_count['target']

In [88]:
target_count_list = y_labels_count['row_count']

In [89]:
recall_list=recall_score(y_test_part, df_top3.iloc[:, 0], labels=y_labels, average=None)
recall_list

array([0.87116741, 0.83622829, 0.88039038, 0.8031514 , 0.93615529,
       0.95701357, 0.80751415, 0.95918926, 0.90857143, 0.97206195,
       0.66300485, 0.76463512, 0.6587202 , 0.70645611, 0.86647315,
       0.80606996, 0.58094207, 0.83138334, 0.60985915, 0.8052223 ,
       0.42921687, 0.79213003, 0.90611542, 0.63116883, 0.96126761,
       0.70416298, 0.87453875, 0.26019846, 0.92713004, 0.77906977,
       0.67380952, 0.18935644, 0.25304465, 0.43502825, 0.34942085,
       0.30416667, 0.19191919, 0.46329114, 0.7989418 , 0.48511905,
       0.66770186, 0.84775087, 0.52830189, 0.92592593, 0.19480519,
       0.12903226, 0.89473684, 0.02116402, 0.22222222, 0.4180791 ,
       0.84848485, 0.03703704, 0.25619835, 0.03389831, 0.        ,
       0.02803738, 0.16842105, 0.28235294, 0.51851852, 0.22222222,
       0.48076923, 0.31111111, 0.625     , 0.35      , 0.05      ,
       0.33333333, 0.07894737, 0.2972973 , 0.27027027, 0.38888889,
       0.19444444, 0.7       , 0.56666667, 0.66666667, 0.37931

In [90]:
precision_list=precision_score(y_test_part, df_top3.iloc[:, 0], labels=y_labels, average=None)
precision_list

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([0.8235134 , 0.70872766, 0.8318679 , 0.75439725, 0.94812885,
       0.97916667, 0.78705794, 0.97739325, 0.90449102, 0.95438283,
       0.48191639, 0.71183277, 0.61070182, 0.77051672, 0.81077411,
       0.55527994, 0.55337803, 0.81346801, 0.68350434, 0.74090909,
       0.55125725, 0.67739576, 0.87302905, 0.74160732, 0.97326203,
       0.57986871, 0.91860465, 0.55399061, 0.92921348, 0.83854819,
       0.71736375, 0.56877323, 0.56666667, 0.5       , 0.61355932,
       0.54275093, 0.50666667, 0.7721519 , 0.80748663, 0.69067797,
       0.74394464, 0.86879433, 0.50179211, 0.85551331, 0.41284404,
       0.31818182, 0.94444444, 0.33333333, 0.61538462, 0.3718593 ,
       0.88607595, 0.38461538, 0.62      , 0.14285714, 0.        ,
       0.25      , 0.48484848, 0.55813953, 0.56756757, 0.31818182,
       0.5952381 , 0.4516129 , 0.96153846, 1.        , 0.2       ,
       1.        , 0.6       , 0.91666667, 0.90909091, 0.93333333,
       1.        , 1.        , 1.        , 0.95238095, 0.64705

In [91]:
f1_list=f1_score(y_test_part, df_top3.iloc[:, 0], labels=y_labels, average=None)
f1_list

array([0.8466704 , 0.76721685, 0.85544161, 0.77801128, 0.94210402,
       0.96796339, 0.79715483, 0.9682057 , 0.90652663, 0.96314127,
       0.55813953, 0.73728977, 0.63380282, 0.73709717, 0.83769878,
       0.65757449, 0.56682515, 0.82232811, 0.64458504, 0.7717281 ,
       0.48264183, 0.73028391, 0.88926458, 0.68194574, 0.96722764,
       0.636     , 0.89603025, 0.35408852, 0.92817059, 0.80771549,
       0.69490485, 0.28412256, 0.34985968, 0.4652568 , 0.44526445,
       0.38985314, 0.27838828, 0.57911392, 0.80319149, 0.56993007,
       0.70376432, 0.85814361, 0.51470588, 0.88932806, 0.26470588,
       0.18360656, 0.91891892, 0.039801  , 0.32653061, 0.39361702,
       0.86687307, 0.06756757, 0.3625731 , 0.05479452, 0.        ,
       0.05042017, 0.25      , 0.375     , 0.54193548, 0.26168224,
       0.53191489, 0.36842105, 0.75757576, 0.51851852, 0.08      ,
       0.5       , 0.13953488, 0.44897959, 0.41666667, 0.54901961,
       0.3255814 , 0.82352941, 0.72340426, 0.78431373, 0.47826

In [92]:
report_df = pd.DataFrame(np.array([y_labels, target_count_list, recall_list, precision_list, f1_list]).T, columns=['target', 'target_count', 'recall', 'precision', 'f1-score'])

In [93]:
report_df.to_csv('Full_train_set_tfidf_xgb_model_eval_119_parts.csv', index=False)